In [ ]:
def register_update_tabs_callback(app, name):
    @app.callback(
        Output('file_tabs' + name, 'tabs'),
        Output('file_tabs' + name, 'children'),
        Input('uploaded-file' + name, 'data'),
        Input('file_tabs' + name, 'active_tab'),
    )
    def update_tabs(data, active_tab):
        """
        We build dynamic tabs based on the files. For each tab, we show
        an <iframe> (or <object>) to display the PDF or text.
        """
        if not data or not data[0]:
            return [], []

        encoded_files, file_texts, file_names = data
        num_files = len(encoded_files)

        # Build a list of tab definitions
        tabs = [
            {'id': f'tab{i + 1}{name}', 'title': file_names[i]}
            for i in range(num_files)
        ]

        # If no active tab, default to first tab
        if not active_tab:
            active_tab = f'tab1{name}'

        # Get the index of the active tab
        try:
            # Extract the number from the tab ID (e.g., 'tab2AWM' -> 2)
            tab_num = int(''.join(filter(str.isdigit, active_tab)))
            tab_id = tab_num - 1  # Convert to 0-based index
        except (ValueError, TypeError):
            tab_id = 0

        # Ensure tab_id is valid
        tab_id = max(0, min(tab_id, len(encoded_files) - 1))

        # Create iframe for the active tab
        children = [
            html.Div([
                html.Iframe(
                    id=f'pdf-viewer-{tab_id}{name}',
                    src=encoded_files[tab_id],
                    style={
                        'width': '100%',
                        'height': '800px',
                        'border': 'none',
                        'marginTop': '10px'
                    }
                )
            ], style={
                'width': '100%',
                'height': '850px',
                'padding': '20px',
                'backgroundColor': '#ffffff'
            })
        ]

        return tabs, children

In [ ]:
def register_update_tabs_callback(app, name):
    @app.callback(
        Output('file_tabs' + name, 'tabs'),
        Output('file_tabs' + name, 'children'),
        Input('uploaded-file' + name, 'data'),
        Input('file_tabs' + name, 'active_tab'),
    )
    def update_tabs(data, active_tab):
        if not data or not data[0]:
            return [], []

        encoded_files, file_texts, file_names = data
        num_files = len(encoded_files)

        tabs = [
            {'id': f'tab{i + 1}{name}', 'title': file_names[i]}
            for i in range(num_files)
        ]

        if not active_tab:
            active_tab = f'tab1{name}'

        try:
            tab_num = int(''.join(filter(str.isdigit, active_tab)))
            tab_id = tab_num - 1
        except (ValueError, TypeError):
            tab_id = 0

        tab_id = max(0, min(tab_id, len(encoded_files) - 1))

        # Create container with JavaScript to handle PDF display
        children = [
            html.Div([
                html.Iframe(
                    id=f'pdf-viewer-{tab_id}{name}',
                    style={
                        'width': '100%',
                        'height': '800px',
                        'border': 'none'
                    }
                ),
                # Store the PDF data
                html.Div(
                    encoded_files[tab_id],
                    id=f'pdf-data-{tab_id}{name}',
                    style={'display': 'none'}
                ),
                # JavaScript to handle PDF display
                html.Script(f'''
                    const pdfData = document.getElementById('pdf-data-{tab_id}{name}').textContent;
                    const base64Data = pdfData.split(',')[1];
                    const byteCharacters = atob(base64Data);
                    const byteArray = new Uint8Array(byteCharacters.length);
                    
                    for (let i = 0; i < byteCharacters.length; i++) {{
                        byteArray[i] = byteCharacters.charCodeAt(i);
                    }}
                    
                    const blob = new Blob([byteArray], {{ type: 'application/pdf' }});
                    const url = URL.createObjectURL(blob);
                    
                    document.getElementById('pdf-viewer-{tab_id}{name}').src = url;
                ''')
            ], style={
                'width': '100%',
                'height': '850px',
                'padding': '20px',
                'backgroundColor': '#ffffff'
            })
        ]

        return tabs, children